In [ ]:
import os
import numpy as np
import scipy as sp
import Configs

# plotting lib(s) and updated default plot settings
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

%matplotlib widget

In [ ]:
EXPERIMENT = 'MNIST_FF'
EXP_CONFIG = getattr(Configs, "Config_{}".format(EXPERIMENT))()
NUM_STEPS_PRED = EXP_CONFIG.NUM_EPOCHS_PHASE2
NUM_EXP_TO_AGG = len([f for f in os.listdir(EXPERIMENT) if 'npz' in f])
WEIGHT_POINTS = (NUM_STEPS_PRED//50) # weights from every 50th step are saved

### Identify Logged Metrics

In [ ]:

experiment_idx = 0
fpath = os.path.join(EXPERIMENT, 'logs_{}_exp_{}.npz'.format(EXPERIMENT, experiment_idx))
lg = np.load(fpath)



In [ ]:
sorted(list(lg.keys()))

In [ ]:
keys_to_plot = ['angle_bw_gradients', 'param_norms', 'train_loss', 'train_accuracy', 'test_loss', 'test_accuracy']
key_labels = ['Angle b/w Grdients', 'Parameter Norms', 'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']
key_ylabels = ['Degrees', 'Squared Norm Units', 'Loss', 'Accuracy(%)', 'Test Loss', 'Test Accuracy(%)']

### Generate Combo Plots

In [ ]:
def generate_combo_plot(metrics, transformation = None, start_exp_id = 0, num_exp_to_agg = NUM_EXP_TO_AGG, step_range=range(NUM_STEPS_PRED)):
    fig, axs = plt.subplots(len(metrics), 1, figsize=(4, 4.5), sharex=True)
    
    combo_plot_data = []
    for metric_index, metric in enumerate(metrics, start=0):
        temp = []

        for experiment_id in range(start_exp_id, start_exp_id+num_exp_to_agg):
            fpath = os.path.join(EXPERIMENT, 'logs_{}_exp_{}.npz'.format(EXPERIMENT, experiment_id))
            lg = np.load(fpath)
            #print(list(lg.keys()))
            #print(metrics[metric_index], len(lg.get(metrics[metric_index])))
            #print(lg.get(metric)[step_range])
            temp.append(lg.get(metric)[-NUM_STEPS_PRED:])


        if transformation is not None:
            temp = [transformation(i) for i in temp]


        mean = np.mean(temp, axis=0)
        std = np.std(temp, axis=0)
        combo_plot_data.append([mean, std])

        axs[metric_index].plot(step_range, mean, label = key_labels[keys_to_plot.index(metric)])
        axs[metric_index].fill_between(step_range, mean+std, mean-std, alpha=0.2)
        axs[metric_index].legend(loc='best')
        #axs[metric_index].set_ylabel(key_ylabels[keys_to_plot.index(metric)], fontsize=9)
        axs[metric_index].set_ylabel("\n".join((key_ylabels[keys_to_plot.index(metric)]).split(" ")), fontsize=9)
        axs[metric_index].legend(loc='best', prop=dict(size=9))
    fig.gca().set_xlabel('Index - Predictor Step')
    plt.tight_layout(h_pad=0.1)
    return fig, combo_plot_data

In [ ]:
fig, data = generate_combo_plot(np.array(keys_to_plot)[[0, 1, -2, -1]])


In [ ]:
fig.savefig('fig_{}.pdf'.format(EXPERIMENT), bbox_inches='tight', dpi=300)

In [ ]:
np.save('{}_combo_plots_data'.format(EXPERIMENT), np.array(data))

### Generate Metris Stats

In [ ]:
def compute_metric(metric, trans = [np.mean, np.std], start_exp_id = 0, num_exp_to_agg = NUM_EXP_TO_AGG, step_range=range(0, NUM_STEPS_PRED)):
    
    
    temp = []

    for exp_index in range(start_exp_id, start_exp_id+num_exp_to_agg):
        fpath = os.path.join(EXPERIMENT, 'logs_{}_exp_{}.npz'.format(EXPERIMENT, exp_index))
        lg = np.load(fpath)
        temp.append(lg.get(metric)[-1])

    return [fn(temp, axis=0) for fn in trans]             


In [ ]:
compute_metric('test_loss'), compute_metric('test_accuracy')

### Extract Traversal Points

In [ ]:

def get_points_traversal(num_trvrsls=None):
    npdumps = sorted([f for f in os.listdir(EXPERIMENT) if 'npz' in f], key=lambda x:int(x.split('_')[-1].split('.')[-2]))
    if num_trvrsls is not None:
        npdumps = npdumps[:num_trvrsls]

    points = []
    for dump in npdumps:
        fpath = os.path.join(EXPERIMENT, dump)
        lg = np.load(fpath)
        for pt in lg.get('weights_phase2')[-WEIGHT_POINTS:]:
            points.append(pt)
            
    points = np.vstack(points)
    
    return points


In [ ]:
pts = get_points_traversal()

In [ ]:
np.save('{}_all_traversal_points'.format(EXPERIMENT), pts)

In [ ]:

def get_avg_endpoint_traversal(num_trvrsls=None):
    npdumps = sorted([f for f in os.listdir(EXPERIMENT) if 'npz' in f], key=lambda x:int(x.split('_')[-1].split('.')[-2]))
    if num_trvrsls is not None:
        npdumps = npdumps[:num_trvrsls]

    points = []
    for dump in npdumps:
        fpath = os.path.join(EXPERIMENT, dump)
        lg = np.load(fpath)
        points.append(lg.get('weights_phase2')[-1])
            
            
    points = np.vstack(points)
    
    return np.mean(points, axis=0)


In [ ]:
avg_endpoint = get_avg_endpoint_traversal()

In [ ]:
np.save('{}_avg_endpoint_traversal'.format(EXPERIMENT), avg_endpoint)

### Inspect Metrics over an Interval

In [ ]:
def generate_focused_plot(metric, transformation = None, start_exp_id = 0, num_exp_to_agg = NUM_EXP_TO_AGG, step_range=range(NUM_STEPS_PRED)):
    plt.figure()
    temp = []


    for experiment_id in range(start_exp_id, start_exp_id+num_exp_to_agg):
        fpath = os.path.join(EXPERIMENT, 'logs_{}_exp_{}.npz'.format(EXPERIMENT, experiment_id))
        lg = np.load(fpath)
        temp.append(lg.get(metric)[step_range])
        
    
    if transformation is not None:
        temp = [transformation(i) for i in temp]
    
    
    mean = np.mean(temp, axis=0)
    std = np.std(temp, axis=0)

    plt.plot(step_range, mean)
    plt.fill_between(step_range, mean+std, mean-std, alpha=0.2)
    
    return plt

In [ ]:

metric = 'angle_bw_gradients'

generate_focused_plot(metric, step_range=range(0, 50))
plt.xlabel('Index - Predictor Step')
plt.ylabel('Degrees')
plt.show()
